In [ ]:
import ollama


def add_two_numbers(a: int, b: int) -> int:
    """
    Add two numbers

    Args:
      a: The first integer number
      b: The second integer number

    Returns:
      int: The sum of the two numbers
    """
    return a + b


response = ollama.chat(
    "llama3.1",
    messages=[{"role": "user", "content": "What is 10 + 10?"}],
    # tools=[add_two_numbers],  # Actual function reference
)

print(response.message.content)

In [ ]:
print(f"Listing available models...")
print(ollama.list())

In [6]:
from typing import Union
from pydantic import BaseModel
import json
import traceback

DEFAULT_SYSTEM_PROMPT = (
    "You are an intelligent assistant. You are helping the user with their query."
)
DEFAULT_TEMPERATURE = 0.5
DEFAULT_MAX_TOKENS = 100
DEFAULT_OLLAMA_MODEL = "llama3.1"
DEFAULT_VERBOSE = True
DEFAULT_DEBUG = True


def generate_object(
    prompt: str,
    response_model: BaseModel,
    system=DEFAULT_SYSTEM_PROMPT,
    model=DEFAULT_OLLAMA_MODEL,
    temperature=DEFAULT_TEMPERATURE,
    max_tokens=DEFAULT_MAX_TOKENS,
    debug=DEFAULT_DEBUG,
    verbose=DEFAULT_VERBOSE,
) -> Union[BaseModel, None]:
    """Generates an object using the OpenAI API and given response model."""
    try:
        if verbose or debug:
            print(f"Generating object for prompt: {prompt}")

        prompt_with_structured_output = f"""
            Prompt: {prompt} 

            Response Schema: '''{response_model.model_json_schema()}'''

            INSTRUCTIONS:
            - STRICTLY RESPOND IN THE GIVEN SCHEMA FORMAT.
            - IT IS MANDATORY TO FOLLOW THE SCHEMA.
            - MAKE SURE TO FILL IN ALL THE REQUIRED FIELDS.
            - DON'T HALLUCINATE OR MAKE UP ANY INFORMATION.
            - RESPOND ONLY WITH VALID JSON AND DON'T INCLUDE ANYTHING ELSE.
        """

        if debug:
            params = {
                "prompt": prompt_with_structured_output,
                "system": system,
                "temperature": temperature,
                "max_tokens": max_tokens,
                "model": model,
            }
            params = json.dumps(params, indent=2)
            print(f"Params: {params}")

        response = ollama.chat(
            model=model,
            messages=[
                {"role": "system", "content": system},
                {"role": "user", "content": prompt},
            ],
        )

        response_json = response.message.content  # Get the response content
        print(response_json)
        response_obj = json.loads(
            response_json
        )  # Convert the response to a Python object

        response_structured = response_model.model_validate(
            response_obj
        )  # Parse the response to the response model

        if verbose or debug:
            print("Object generated successfully. 🎉")

        if debug:
            print(f"EasyLLM Response: {response_json}")
        return response_structured
    except Exception as e:
        print(f"Failed to generate object. Error: {str(e)}")
        if debug:
            traceback.print_exc()
        return None

In [ ]:
from pydantic import BaseModel


class Joke(BaseModel):
    joke: str = (
        "Why did the scarecrow win an award? Because he was outstanding in his field."
    )
    author: str = "Unknown"


response = generate_object("Tell me a joke", response_model=Joke)

print(response)

In [ ]:
import ollama

from pydantic import BaseModel


class Joke(BaseModel):
    joke: str
    author: str


prompt = f"""
"Tell me a joke. 
Respond with the schema: {Joke.model_json_schema()}"
Include only the keys and values and not the schema itself.
"""

response = ollama.chat(
    "phi4",
    messages=[
        {
            "role": "user",
            "content": f"Tell me a joke. Respond with the schema: {Joke.model_json_schema()}",
        }
    ],
)

print(response.message.content)

In [ ]:
import ollama

from pydantic import BaseModel


class Joke(BaseModel):
    joke: str = "A joke"
    author: str = "An author"


def build_dummy_pydantic_object(schema: BaseModel) -> BaseModel:
    """
    Build a dummy Pydantic object using the given schema.

    Args:
      schema: The Pydantic schema to build the object from

    Returns:
      BaseModel: The dummy Pydantic object
    """
    return schema()


prompt = f"""
"Tell me a joke. 
Respond with the schema — Example object: {build_dummy_pydantic_object().model_dump_json()}"
Include only the keys and values and not the schema itself.
Respond strictly in the given schema format.
Respond only with valid JSON and don't include anything else.
"""

response = ollama.chat("llama3.1", messages=[{"role": "user", "content": prompt}])

print(response.message.content)